In [15]:
import json
import pandas as pd

data = [ "train", "test" ]

#Read jsons and combine to concatenated dataframe
df = pd.concat([pd.read_json( f"{d}.json" ) for d in data])
df.head()

,utterance,slots,intent
0,i want to fly from boston at 838 am and arrive...,O O O O O B-fromloc.city_name O B-depart_time....,flight
1,what flights are available from pittsburgh to ...,O O O O O B-fromloc.city_name O B-toloc.city_n...,flight
2,what is the arrival time in san francisco for ...,O O O B-flight_time I-flight_time O B-fromloc....,flight_time
3,cheapest airfare from tacoma to orlando,B-cost_relative O O B-fromloc.city_name O B-to...,airfare
4,round trip fares from pittsburgh to philadelph...,B-round_trip I-round_trip O O B-fromloc.city_n...,airfare


In [16]:
#drop 'slots'
df.drop('slots', axis=1, inplace=True)

def correct_label( row ):
  return " ".join( [word.capitalize() for word in row.split("_")] ) + "."

df['intent'] = df['intent'].apply( correct_label )

df.head()

,utterance,intent
0,i want to fly from boston at 838 am and arrive...,Flight.
1,what flights are available from pittsburgh to ...,Flight.
2,what is the arrival time in san francisco for ...,Flight Time.
3,cheapest airfare from tacoma to orlando,Airfare.
4,round trip fares from pittsburgh to philadelph...,Airfare.


In [17]:
df.to_csv( "data.csv" )